# Imports

In [1]:
import cs7641assn4 as a4
import numpy as np
import pandas as pd
import warnings

pd.set_option('display.max_columns', 35)
# pd.reset_option("display.max_columns")

# Establish Environment

In [2]:
rH = -1 #-5 # reward for H(ole)
rG = 1 # 10 # reward for G(oal)
rF = -0.2# reward includes S(tart) and F(rozen)
size = 8 # height and width of square gridworld
p = 0.8 # if generating a random map probability that a grid will be F(rozen)
desc = None # frozen_lake.generate_random_map(size=size, p=p)
map_name = 'x'.join([str(size)]*2) # None
is_slippery = True


epsilon = 1e-8 # convergence threshold for policy/value iteration
gamma = 0.8 # discount parameter for past policy/value iterations
max_iter = 10000 # maximum iterations for slowly converging policy/value iteration 

# Qlearning(env, rH=0, rG=1, rF=0, qepsilon=0.1, lr=0.8, gamma=0.95, episodes=10000)
qepsilon = 0.1 # epsilon value for the Q-learning epsilon greedy strategy
lr = 0.8 # Q-learning rate
qgamma = 0.95 # Q-Learning discount factor
episodes = 10000 # number of Q-learning episodes
initial = 0 # value to initialize the Q grid

# Create Environment
env = a4.getEnv(env_id='hw4-FrozenLake-v0', rH=rH, rG=rG, rF=rF, 
                desc=desc, map_name=map_name, 
                is_slippery=is_slippery,render_initial=True)

# Store a representation of the map
env_desc = env.desc.astype('<U8')

# Store a representation of the state rewards
env_rs = a4.getStateReward(env)

# Display reward at each state
print('\n--Reward Values at Each State--')
a4.matprint(a4.print_value(env_rs, width=size, height=size))

--Board--

S  F  F  F  F  F  F  F
F  F  F  F  F  F  F  F
F  F  F  H  F  F  F  F
F  F  F  F  F  H  F  F
F  F  F  H  F  F  F  F
F  H  H  F  F  F  H  F
F  H  F  F  H  F  H  F
F  F  F  H  F  F  F  G

--Actions for Position to the Left of the Goal--
{0: [(0.3333333333333333, 54, -1, True),
     (0.3333333333333333, 61, -0.2, False),
     (0.3333333333333333, 62, -0.2, False)],
 1: [(0.3333333333333333, 61, -0.2, False),
     (0.3333333333333333, 62, -0.2, False),
     (0.3333333333333333, 63, 1, True)],
 2: [(0.3333333333333333, 62, -0.2, False),
     (0.3333333333333333, 63, 1, True),
     (0.3333333333333333, 54, -1, True)],
 3: [(0.3333333333333333, 63, 1, True),
     (0.3333333333333333, 54, -1, True),
     (0.3333333333333333, 61, -0.2, False)]}

--Reward Values at Each State--
-0.2  -0.2  -0.2  -0.2  -0.2  -0.2  -0.2  -0.2  
-0.2  -0.2  -0.2  -0.2  -0.2  -0.2  -0.2  -0.2  
-0.2  -0.2  -0.2    -1  -0.2  -0.2  -0.2  -0.2  
-0.2  -0.2  -0.2  -0.2  -0.2    -1  -0.2  -0.2  
-0.2  -0.2  -0.

# Policy Iteration

In [3]:
pi_time = %timeit -o a4.policy_iteration(env, epsilon=epsilon, gamma=gamma, max_iter=max_iter, report=False)

8.42 ms ± 1.14 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [4]:
pi_V, pi_policy, pi_epochs = a4.policy_iteration(env, epsilon=epsilon, gamma=gamma, max_iter=max_iter, report=True)

# Display values
a4.matprint(a4.print_value(pi_V, width=size, height=size))

pi_policy_arrows = a4.print_policy(pi_policy, width=size, height=size)

# Display policy
a4.matprint(pi_policy_arrows)

Policy iteration converged after  7 epochs
-0.9992  -0.9987  -0.9974   -0.995  -0.9904  -0.9824  -0.9707  -0.9647  
-0.9993  -0.9989  -0.9979  -0.9958  -0.9912  -0.9809  -0.9548  -0.9382  
-0.9997  -1.0347   -1.141       -5  -1.1636  -1.0177  -0.9217  -0.8751  
-0.9999  -1.1315  -1.4961  -2.6269  -1.6048       -5  -0.8763   -0.735  
-0.9999  -1.4585  -2.5879       -5  -1.4774  -1.2764  -0.8797   -0.396  
     -1       -5       -5  -3.2524  -1.9091  -1.6793       -5    0.396  
     -1       -5  -3.2628  -3.5374       -5  -2.3619       -5   2.2349  
     -1  -1.3448  -1.9482       -5  -2.6101  -0.4278   1.9353        5  
↑  →  →  →  →  →  →  →  
↑  ↑  ↑  ↑  ↑  ↑  →  ↓  
←  ←  ←  ←  →  ↑  →  ↓  
←  ←  ←  ↓  ←  ←  →  →  
←  ↑  ←  ←  →  ↓  ↑  →  
←  ←  ←  ↓  ↑  ←  ←  →  
←  ←  ↓  ↑  ←  ←  ←  →  
←  ↓  ←  ←  →  →  ↓  ←  


# Value Iteration

In [5]:
vi_time = %timeit -o a4.valueIteration(env, epsilon=epsilon, gamma=gamma, max_iter=max_iter, report=False)

38.5 ms ± 7.09 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [6]:
vi_V, vi_epochs = a4.valueIteration(env, epsilon=epsilon, gamma=gamma, max_iter=max_iter, report=True)

# display value function:
a4.matprint(a4.print_value(vi_V, width=size, height=size))

vi_policy = a4.value_to_policy(env, V=vi_V, gamma=gamma)

vi_policy_arrows = a4.print_policy(vi_policy, width=size, height=size)
# display policy
a4.matprint(vi_policy_arrows)

Value iteration converged after  91 epochs
-0.9992  -0.9987  -0.9974   -0.995  -0.9904  -0.9824  -0.9707  -0.9647  
-0.9993  -0.9989  -0.9979  -0.9958  -0.9912  -0.9809  -0.9548  -0.9382  
-0.9997  -1.0347   -1.141       -5  -1.1636  -1.0177  -0.9217  -0.8751  
-0.9999  -1.1315  -1.4961  -2.6269  -1.6048       -5  -0.8763   -0.735  
-0.9999  -1.4585  -2.5879       -5  -1.4774  -1.2764  -0.8797   -0.396  
     -1       -5       -5  -3.2524  -1.9091  -1.6793       -5    0.396  
     -1       -5  -3.2628  -3.5374       -5  -2.3619       -5   2.2349  
     -1  -1.3448  -1.9482       -5  -2.6101  -0.4278   1.9353        5  
↑  →  →  →  →  →  →  →  
↑  ↑  ↑  ↑  ↑  ↑  →  ↓  
←  ←  ←  ←  →  ↑  →  ↓  
←  ←  ←  ↓  ←  ←  →  →  
←  ↑  ←  ←  →  ↓  ↑  →  
←  ←  ←  →  ↑  ←  ←  →  
←  ←  ↓  ←  ←  ←  ←  →  
←  ↓  ←  ←  ↓  →  ↓  ←  


# Q-Learning

In [7]:
Q_time = %timeit -o a4.Qlearning(env, qepsilon, lr, qgamma, episodes)

5.82 s ± 535 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [8]:
Q = a4.Qlearning(env, qepsilon, lr, qgamma, episodes)
print('--Q with all options--')
a4.matprint(Q)

maxQ = np.max(Q,axis=1)
print('\n--argmax(Q) in grid order--')
a4.matprint(a4.print_value(maxQ, width=size, height=size))

Q_policy = a4.Q_to_policy(Q)

Q_policy_arrows = a4.print_policy(Q_policy, width=size, height=size)
print('\n--Policy Matrix--')
a4.matprint(Q_policy_arrows)

--Q with all options--
 -2.90483   -2.65097    -2.88639   -2.95405  
 -2.82961   -2.88577    -2.67045   -2.94525  
 -2.88789   -2.82255    -2.60516   -2.76892  
 -2.81737   -2.66812    -2.59523   -2.65026  
 -2.63818   -2.79209    -2.71356   -2.79497  
 -2.81337   -2.70316    -2.59647   -2.85473  
 -2.76219   -2.73693    -2.53896   -2.84403  
 -2.68181   -2.69991    -2.71242   -2.62903  
 -2.84509   -2.58098    -2.91415   -2.85357  
 -2.67057   -2.86058    -2.46926   -2.86612  
 -2.75019   -2.90422    -2.16663   -2.75262  
 -2.62764   -2.56064    -2.73061   -2.76128  
 -2.13244   -2.52978    -2.50001   -2.49557  
 -2.68237   -2.49771    -2.53835   -2.61581  
 -2.74122   -2.21025    -2.63426   -2.79655  
 -2.66316   -2.48514     -2.8871   -2.91094  
 -2.54518   -2.53061     -2.9273   -2.92271  
  -2.7764   -2.81393     -2.4725   -2.86125  
  -2.5876   -2.46215    -1.96077   -2.66429  
       -1         -1          -1         -1  
 -2.18055   -2.37197    -2.40507   -2.32767  
 -2.52141  

In [9]:
Q_s, Q_steps = a4.Qlearning_trajectory(env, Q, render=False)


--Final position--
  (Left)
S  F  F  F  F  F  F  F
F  F  F  F  F  F  F  F
F  F  F  H  F  F  F  F
F  F  F  F  F  H  F  F
F  F  F  H  F  F  F  F
F  H  H  F  F  F  H  F
F  H  F  F  H  F  H  F
F  F  F  H  F  F  F  G
Agent ended up at state 19 after 13 steps


# Notes

Default rewards are 1 for the G(oal) and 0 for everything else.

Maps are drawn according to the following logic

```
if desc and map_name are None, 
   then a default random map is drawn with 8
        using frozen_lake.generate_random_map(size=8, p=0.8)
elif desc is None and a map_name is given
   then a map_name is either '4x4' or '8x8'
        and is drawn from the dict MAPS in frozen_lake.py
elif desc is given
   then it must be in the form of a list with 
```

Default action probabilities are 1/3 chosen action, 1/3 each for right angles to chosen action, and 0 for reverse of chosen action. This is set with `is_slippery=True`. If `is_slippery=False`, then P=1 for chosen action and 0 for all other actions.

|ACTION|Value|Symbol|
|------|-----|------|
|LEFT  | 0   | ←    |
|DOWN  | 1   | ↓    |
|RIGHT | 2   | →    |
|UP    | 3   | ↑    |

# Sources

- Code: <https://github.com/Twice22/HandsOnRL>
- Tutorial: <https://twice22.github.io/>

# Save Results to DataFrame

In [10]:
results = pd.DataFrame({'rH': [rH], 
                        'rG': [rG], 
                        'rF': [rF], 
                        'size': [size], 
                        'p': [p], 
                        'desc': [desc], 
                        'map_name': [map_name],                        
                        'is_slippery': [is_slippery],
                        'epsilon': [epsilon],
                        'gamma': [gamma], 
                        'max_iter': [max_iter], 
                        'qepsilon': [qepsilon], 
                        'lr': [lr], 
                        'qgamma': [qgamma], 
                        'episodes': [episodes], 
                        'initial': [initial],
                        'env_desc': [env_desc],
                        'env_rs': [env_rs],
                        'pi_time': [pi_time.average],
                        'pi_V': [pi_V],
                        'pi_epochs': [pi_epochs],
                        'pi_policy': [pi_policy],
                        'pi_policy_arrows': [pi_policy_arrows],
                        'vi_time': [vi_time.average],
                        'vi_V': [vi_V],
                        'vi_epochs': [vi_epochs],
                        'vi_policy': [vi_policy],
                        'vi_policy_arrows': [vi_policy_arrows],
                        'Q_time': [Q_time.average],
                        'Q': [Q],
                        'Q_V': [maxQ],
                        'Q_policy': [Q_policy],
                        'Q_policy_arrows': [Q_policy_arrows]})

In [11]:
display(results)

,rH,rG,rF,size,p,desc,map_name,is_slippery,epsilon,gamma,max_iter,qepsilon,lr,qgamma,episodes,initial,env_desc,env_rs,pi_time,pi_V,pi_epochs,pi_policy,pi_policy_arrows,vi_time,vi_V,vi_epochs,vi_policy,vi_policy_arrows,Q_time,Q,Q_V,Q_policy,Q_policy_arrows
0,-1,1,-0.2,8,0.8,None,8x8,True,1.000000e-08,0.8,10000,0.1,0.8,0.95,10000,0,"[[S, F, F, F, F, F, F, F], [F, F, F, F, F, F, ...","[-0.2, -0.2, -0.2, -0.2, -0.2, -0.2, -0.2, -0....",0.008424,"[-0.9992295366480467, -0.9986516891340815, -0....",7,"[3, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 2, ...","[[↑, →, →, →, →, →, →, →], [↑, ↑, ↑, ↑, ↑, ↑, ...",0.038484,"[-0.999229537127132, -0.9986516896132036, -0.9...",91,"[3, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 2, ...","[[↑, →, →, →, →, →, →, →], [↑, ↑, ↑, ↑, ↑, ↑, ...",5.819217,"[[-2.904827942229138, -2.6509684076844677, -2....","[-2.6509684076844677, -2.67045146734101, -2.60...","[1, 2, 2, 2, 0, 2, 2, 3, 1, 2, 2, 1, 0, 1, 1, ...","[[↓, →, →, →, ←, →, →, ↑], [↓, →, →, ↓, ←, ↓, ..."


# Save DataFrame to Disk

In [12]:
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)
try:
    dataset = pd.read_hdf('data.h5', key='dataset', mode='a')
except FileNotFoundError:
    results.to_hdf('data.h5', key='dataset', mode='a')
else:
    dataset.append(
        other=results, 
        ignore_index=True
        ).to_hdf(
        path_or_buf='data.h5', 
        key='dataset', 
        mode='a')

In [13]:
pd.read_hdf('data.h5', key='dataset', mode='a')

,rH,rG,rF,size,p,desc,map_name,is_slippery,epsilon,gamma,max_iter,qepsilon,lr,qgamma,episodes,initial,env_desc,env_rs,pi_time,pi_V,pi_epochs,pi_policy,pi_policy_arrows,vi_time,vi_V,vi_epochs,vi_policy,vi_policy_arrows,Q_time,Q,Q_V,Q_policy,Q_policy_arrows
0,-1,1,-0.2,4,0.8,None,4x4,False,1.000000e-08,0.8,10000,0.1,0.8,0.95,10000,1,"[[S, F, F, F], [F, H, F, H], [F, F, F, H], [H,...","[-0.2, -0.2, -0.2, -0.2, -0.2, -1.0, -0.2, -1....",0.001161,"[0.9660800000000008, 1.457600000000001, 2.0720...",7,"[1, 2, 1, 0, 1, 0, 1, 0, 2, 1, 1, 0, 0, 2, 2, 0]","[[↓, →, ↓, ←], [↓, ←, ↓, ←], [→, ↓, ↓, ←], [←,...",0.008001,"[0.9660799905143127, 1.4575999905143127, 2.071...",91,"[1, 2, 1, 0, 1, 0, 1, 0, 2, 1, 1, 0, 0, 2, 2, 0]","[[↓, →, ↓, ←], [↓, ←, ↓, ←], [→, ↓, ↓, ←], [←,...",1.619264,"[[0.37379674921874956, 0.6039965781249996, 0.6...","[0.6039965781249996, 0.8463121874999997, 1.101...","[1, 2, 1, 0, 1, 0, 1, 0, 2, 1, 1, 0, 0, 2, 2, 0]","[[↓, →, ↓, ←], [↓, ←, ↓, ←], [→, ↓, ↓, ←], [←,..."
1,-1,1,-0.2,8,0.8,None,8x8,False,1.000000e-08,0.8,10000,0.1,0.8,0.95,10000,1,"[[S, F, F, F, F, F, F, F], [F, F, F, F, F, F, ...","[-0.2, -0.2, -0.2, -0.2, -0.2, -0.2, -0.2, -0....",0.008319,"[-0.6701465116672001, -0.5876831395840001, -0....",13,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 2, 1, 1, 1, ...","[[↓, ↓, ↓, ↓, ↓, ↓, ↓, ↓], [↓, →, ↓, →, ↓, ↓, ...",0.032797,"[-0.6701465165238721, -0.587683144440672, -0.4...",94,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, ...","[[↓, ↓, ↓, ↓, ↓, ↓, ↓, ↓], [↓, ↓, ↓, →, ↓, ↓, ...",3.948933,"[[-1.0983338742625979, -0.9456146044869451, -0...","[-0.9456146044869451, -0.7848574784073106, -0....","[1, 1, 2, 1, 2, 1, 1, 0, 1, 1, 2, 2, 1, 1, 2, ...","[[↓, ↓, →, ↓, →, ↓, ↓, ←], [↓, ↓, →, →, ↓, ↓, ..."
2,-1,1,-0.2,4,0.8,None,4x4,True,1.000000e-08,0.8,10000,0.1,0.8,0.95,10000,1,"[[S, F, F, F], [F, H, F, H], [F, F, F, H], [H,...","[-0.2, -0.2, -0.2, -0.2, -0.2, -1.0, -0.2, -1....",0.000694,"[-0.925710165014909, -0.9675729521710873, -0.9...",4,"[0, 3, 3, 3, 0, 0, 0, 0, 3, 1, 0, 0, 0, 2, 1, 0]","[[←, ↑, ↑, ↑], [←, ←, ←, ←], [↑, ↓, ←, ←], [←,...",0.007028,"[-0.9257101677274406, -0.9675729537982982, -0....",89,"[0, 3, 3, 3, 0, 0, 0, 0, 3, 1, 0, 0, 0, 2, 1, 0]","[[←, ↑, ↑, ↑], [←, ←, ←, ←], [↑, ↓, ←, ←], [←,...",1.484242,"[[-2.412922260811493, -2.211129810113885, -2.3...","[-2.211129810113885, -2.0260349734160794, -2.1...","[1, 0, 2, 0, 2, 0, 1, 0, 1, 2, 3, 0, 0, 1, 2, 0]","[[↓, ←, →, ←], [→, ←, ↓, ←], [↓, →, ↑, ←], [←,..."
3,-1,1,-0.2,8,0.8,None,8x8,True,1.000000e-08,0.8,10000,0.1,0.8,0.95,10000,1,"[[S, F, F, F, F, F, F, F], [F, F, F, F, F, F, ...","[-0.2, -0.2, -0.2, -0.2, -0.2, -0.2, -0.2, -0....",0.006229,"[-0.9992295366480467, -0.9986516891340815, -0....",7,"[3, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 2, ...","[[↑, →, →, →, →, →, →, →], [↑, ↑, ↑, ↑, ↑, ↑, ...",0.028723,"[-0.999229537127132, -0.9986516896132036, -0.9...",91,"[3, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 2, ...","[[↑, →, →, →, →, →, →, →], [↑, ↑, ↑, ↑, ↑, ↑, ...",5.524295,"[[-3.092363894051347, -3.06339133380597, -2.74...","[-2.7458258330363328, -2.737544989590017, -2.8...","[2, 2, 1, 1, 1, 1, 2, 1, 1, 2, 2, 2, 3, 1, 1, ...","[[→, →, ↓, ↓, ↓, ↓, →, ↓], [↓, →, →, →, ↑, ↓, ..."
4,-1,1,-0.2,4,0.8,None,4x4,False,1.000000e-08,0.8,10000,0.1,0.8,0.95,10000,0,"[[S, F, F, F], [F, H, F, H], [F, F, F, H], [H,...","[-0.2, -0.2, -0.2, -0.2, -0.2, -1.0, -0.2, -1....",0.001062,"[0.9660800000000008, 1.457600000000001, 2.0720...",7,"[1, 2, 1, 0, 1, 0, 1, 0, 2, 1, 1, 0, 0, 2, 2, 0]","[[↓, →, ↓, ←], [↓, ←, ↓, ←], [→, ↓, ↓, ←], [←,...",0.007250,"[0.9660799905143127, 1.4575999905143127, 2.071...",91,"[1, 2, 1, 0, 1, 0, 1, 0, 2, 1, 1, 0, 0, 2, 2, 0]","[[↓, →, ↓, ←], [↓, ←, ↓, ←], [→, ↓, ↓, ←], [←,...",1.608711,"[[0.37379674921874956, 0.6039965781249996, 0.6...","[0.6039965781249996, 0.8463121874999997, 1.101...","[1, 2, 1, 0, 1, 0, 1, 0, 2, 1, 1, 0, 0, 2, 2, 0]","[[↓, →, ↓, ←], [↓, ←, ↓, ←], [→, ↓, ↓, ←], [←,..."
5,-1,1,-0.2,8,0.8,None,8x8,False,1.000000e-08,0.8,10000,0.1,0.8,0.95,10000,0,"[[S, F, F, 